# Using ROBOKOP's synonymizer service

Entities in translator are references using URIs. Usually, these are Compact URIs (Curies).   The tricky and annoying thing is that there are many different Curies all pointing at the same real entity.  A disease may have a curie for a DOID or a MONDO identifier, or a UMLS, and so on.  The synonymizer service takes in a curie and returns the list of other identifiers for that entity.

In [1]:
robokop_server = 'robokop.renci.org'

In [2]:
import requests
import json
import pandas as pd

The following python function shows how to call the ROBOKOP synonymizer service.  It takes an identifier, and a biolink model type name.

In [3]:
def synonymize(nodetype,identifier):
    url=f'http://{robokop_server}:6010/api/synonymize/{identifier}/{nodetype}/'
    response = requests.post(url)
    print( f'Return Status: {response.status_code}' )
    if response.status_code == 200:
        return response.json()
    return []

## Basic Usage

In this example, we have the `disease` Fanconi Anemia defined by the curie identifier `MONDO:0019391`.  We want to know the `phenotypic_feature`s that are associated with it.  We can call the function above like this:

In [4]:
other_fanconi_identifiers = synonymize('disease', 'MONDO:0019391')
other_fanconi_identifiers

Return Status: 200


{'id': 'MONDO:0019391',
 'name': '',
 'synonyms': [['MEDDRA:10055206', ''],
  ['DOID:13636', ''],
  ['UMLS:C0015625', ''],
  ['MEDDRA:10016218', ''],
  ['MESH:D005199', ''],
  ['MONDO:0019391', 'Fanconi anemia'],
  ['ORPHANET:84', '']],
 'type': 'disease'}

Note that the object returned contains a list of synonyms. Each synonym is a list containing an identifier and possibly a name (though in many cases, the name string is empty.)

In [5]:
other_caffeine_identifiers = synonymize('chemical_substance', 'CHEBI:27732')
other_caffeine_identifiers

Return Status: 200


{'id': 'CHEBI:27732',
 'name': '',
 'synonyms': [['DRUGBANK:DB00201', ''],
  ['HMDB:HMDB0001847', ''],
  ['KEGG.COMPOUND:C07481', ''],
  ['UNII:3G6A5W338E', ''],
  ['CHEMBL:CHEMBL113', 'CAFFEINE'],
  ['PUBCHEM:2519', ''],
  ['MESH:D002110', 'Caffeine'],
  ['CHEBI:27732', 'caffeine']],
 'type': 'chemical_substance'}

In [6]:
other_gene_identifiers = synonymize('gene', 'HGNC:6342')
other_gene_identifiers

Return Status: 200


{'id': 'HGNC:6342',
 'name': '',
 'synonyms': [['UniProtKB:A0A0U2N547', ''],
  ['HGNC:6342', 'KIT'],
  ['UniProtKB:A0A024RDA0', ''],
  ['UniProtKB:P10721', 'KIT'],
  ['ENSEMBL:ENSG00000157404', 'KIT'],
  ['NCBIGENE:3815', 'KIT']],
 'type': 'gene'}

## Biolink-model hierarchy: The simple case

Superclasses and subclasses of biolink model types are useable.  So we can use `metabolite` instead of `chemical_substance`.

In [7]:
synonymize('metabolite', 'CHEBI:27732')

Return Status: 200


{'id': 'CHEBI:27732',
 'name': '',
 'synonyms': [['DRUGBANK:DB00201', ''],
  ['HMDB:HMDB0001847', ''],
  ['KEGG.COMPOUND:C07481', ''],
  ['UNII:3G6A5W338E', ''],
  ['CHEMBL:CHEMBL113', 'CAFFEINE'],
  ['PUBCHEM:2519', ''],
  ['MESH:D002110', 'Caffeine'],
  ['CHEBI:27732', 'caffeine']],
 'type': 'metabolite'}

We can also use the superclass `disease_or_phenotypic_feature` instead of `disease`.

In [8]:
synonymize('disease_or_phenotypic_feature', 'MONDO:0019391')

Return Status: 200


{'id': 'MONDO:0019391',
 'name': '',
 'synonyms': [['MEDDRA:10055206', ''],
  ['DOID:13636', ''],
  ['UMLS:C0015625', ''],
  ['NCIT:C62505', ''],
  ['MEDDRA:10016218', ''],
  ['MESH:D005199', ''],
  ['MONDO:0019391', 'Fanconi anemia'],
  ['ORPHANET:84', '']],
 'type': 'disease_or_phenotypic_feature'}

## Biolink Model types: a more complicated case

Some care needs to be applied by the user here, because the results that one gets can be type dependent.  For instance, consider ALS, which has a `disease` identifier in MONDO, and a `phenotypic_feature` identifier in HPO.
In biolink model, these are separate entities:

In [9]:
synonymize('disease','MONDO:0004976')

Return Status: 200


{'id': 'MONDO:0004976',
 'name': '',
 'synonyms': [['ORPHANET:803', ''],
  ['DOID:332', ''],
  ['MESH:D000690', ''],
  ['UMLS:C0002736', ''],
  ['MEDDRA:10002026', ''],
  ['MONDO:0004976', 'amyotrophic lateral sclerosis'],
  ['MEDDRA:10052889', '']],
 'type': 'disease'}

In [10]:
synonymize('phenotypic_feature','HP:0007354')

Return Status: 200


{'id': 'HP:0007354',
 'name': '',
 'synonyms': [['NCIT:C34373', ''],
  ['UMLS:C0002736', ''],
  ['HP:0007354', 'Amyotrophic lateral sclerosis'],
  ['MEDDRA:10002026', ''],
  ['MEDDRA:10052889', '']],
 'type': 'phenotypic_feature'}

Note that in both cases, `UMLS:C000736` is returned as a synonym.  That's because a UMLS can be either a `disease` or a `phenotypic_feature`.  But the MONDO is not returned for the `phenotypic_feature` call because a MONDO identifier cannot point to a `phenotypic_feature`.  Similarly, the HP identifier is not returned if we ask for `disease`.   What if we ask for synonyms of `UMLS:C0002736`?   The answer depends on what type we assert we want. If we ask for diseases, we get the MONDO identifier (and not the HP):

In [11]:
synonymize('disease','UMLS:C0002736')

Return Status: 200


{'id': 'MONDO:0004976',
 'name': 'amyotrophic lateral sclerosis',
 'synonyms': [['ORPHANET:803', ''],
  ['DOID:332', ''],
  ['MESH:D000690', ''],
  ['UMLS:C0002736', ''],
  ['MEDDRA:10002026', ''],
  ['MONDO:0004976', 'amyotrophic lateral sclerosis'],
  ['MEDDRA:10052889', '']],
 'type': 'disease'}

If we ask for a `phenotypic_feature` we get the HP (and not the MONDO):

In [12]:
synonymize('phenotypic_feature','UMLS:C0002736')

Return Status: 200


{'id': 'HP:0007354',
 'name': 'Amyotrophic lateral sclerosis',
 'synonyms': [['NCIT:C34373', ''],
  ['UMLS:C0002736', ''],
  ['HP:0007354', 'Amyotrophic lateral sclerosis'],
  ['MEDDRA:10002026', ''],
  ['MEDDRA:10052889', '']],
 'type': 'phenotypic_feature'}

And if we ask for the union superclass `disease_or_phenoptyic_feature`, we get both:

In [13]:
synonymize('disease_or_phenotypic_feature','UMLS:C0002736')

Return Status: 200


{'id': 'MONDO:0004976',
 'name': 'amyotrophic lateral sclerosis',
 'synonyms': [['ORPHANET:803', ''],
  ['NCIT:C34373', ''],
  ['DOID:332', ''],
  ['MESH:D000690', ''],
  ['UMLS:C0002736', ''],
  ['HP:0007354', 'Amyotrophic lateral sclerosis'],
  ['MEDDRA:10002026', ''],
  ['MONDO:0004976', 'amyotrophic lateral sclerosis'],
  ['MEDDRA:10052889', '']],
 'type': 'disease_or_phenotypic_feature'}